In [1]:
import numpy as np
from math import pow
import scipy.optimize

## 1 载入原始数据

### 获取基站个数sta_num（第一行），终端个数dev_num（第二行），数据维度（第三行）

In [2]:
handle_list=[]
with open('./Dataset_2016c/sample_case001_input.txt','r') as handle:
    for line in handle:
        handle_list.append(line.strip('\n'))
sta_num = int(handle_list[0])
dev_num = int(handle_list[1])
dimension = int(handle_list[2])

### 构建基站位置列表

In [3]:
sta_list = []
for i in range(3,3+sta_num):
    sta_list.append([float(x) for x in handle_list[i].split('\t') if x != ''])

### 构建终端TOA列表

In [4]:
toa_list = []
for i in range(3+sta_num,3+sta_num+dev_num):
    toa_list.append([float(x) for x in handle_list[i].split('\t') if x != ''])

## 2 定位基本模型

### 构建矩阵A、X、Y，并用最小二乘法求解最小值
![矩阵](Dataset_2016c/tutorial_img/matrix.png)

In [14]:
def residual(p,mat_A,K_k,L_k):
    mat_Y = []
    count = 0
    for i in range(sta_num):
        L_i = wave_spread_speed * toa_list[k_2][i]
        if is_valid_TOA(L_i):
            K_i = x_i*x_i+y_i*y_i+z_i*z_i
            mat_Y.append(K_i-pow(L_i*p[count+dimension],2)-K_k+pow(L_k*p[dimension],2))
            count += 1
    mat_temp = np.dot(mat_A,[p[0],p[1],p[2]])-mat_Y
    # delta = np.sum(np.multiply(mat_temp,mat_temp))
    delta = float(np.dot(mat_temp.T,mat_temp))
    return delta

def is_valid_TOA(L):
    if L <= float(200/.62):
        return True
    else:
        return False

handle_list=[]
with open('./Dataset_2016c/sample_case001_ans.txt','r') as handle:
    for line in handle:
        handle_list.append(line.strip('\n'))
ans_list = []
for line in handle_list:
    ans_list.append([float(x) for x in line.split('\t') if x != ''])

wave_spread_speed = 3e8
for k_2 in range(dev_num):
    mat_A = []
    for k in range(sta_num):
        L_k = wave_spread_speed * toa_list[k_2][k]
        if is_valid_TOA(L_k):
            x_k,y_k,z_k = sta_list[k][0],sta_list[k][1],sta_list[k][2]
            K_k = x_k*x_k+y_k*y_k+z_k*z_k
            break
    if not (x_k and y_k and z_k):
        print("{}:Bad signal!".format(k_2))
        continue
    for i in range(sta_num):
        L_i = wave_spread_speed * toa_list[k_2][i]
        if is_valid_TOA(L_i): # 假设每个基站的通信半径为200米，考虑到NLOS，除以折减系数（超过范围虽然有测量数据，但无效）
            x_i,y_i,z_i = sta_list[i][0],sta_list[i][1],sta_list[i][2]
            mat_A.append([x_i-x_k,y_i-y_k,z_i-z_k])
    if len(mat_A) < 4:
        print("{}:Not enough valid TOA data! Available: {}".format(k_2,len(mat_A)))
        continue
    # 以TOA最小值对应的基站坐标为初始值
    p_init = sta_list[toa_list[k_2].index(min(toa_list[k_2]))][:]
    cons = ({'type': 'ineq', 'fun': lambda x: 40000-pow(x[0]-p_init[0],2)-pow(x[1]-p_init[1],2)-pow(x[2]-p_init[2],2)})
    bnds = [(None,None),(None,None),(None,None)]
    for i in range(len(mat_A)):
        p_init.append(.85) # φ_i
        bnds.append((0.62,0.98))
    param=scipy.optimize.minimize(residual,p_init,args=(mat_A,K_k,L_k),method='SLSQP',constraints=cons,bounds=bnds)
    print("{}:{} ({}); Answer: {}".format(k_2,np.round(param.x,2)[:3],len(mat_A),ans_list[k_2]))

0:[-57.4   -9.29   4.52] (15); Answer: [-21.19, 4.48, 1.48]
1:[-57.4   -9.29   4.52] (15); Answer: [-81.14, 58.24, 1.38]
2:[ -84.5  -184.89    4.58] (12); Answer: [-96.13, -215.22, 1.02]
3:[-273.67  -21.14    4.98] (6); Answer: [-296.06, -20.15, 1.59]
4:[ 36.29 -85.79   3.19] (14); Answer: [86.29, -111.06, 1.66]
5:[ 20.77 244.49   4.81] (6); Answer: [-31.26, 244.68, 1.3]
6:[-273.67  -21.14    4.98] (6); Answer: [-286.47, -38.75, 1.73]
7:[ 279.66 -128.13    3.98] (5); Answer: [319.3, -170.11, 1.03]
8:[ 20.77 244.49   4.81] (6); Answer: [-37.4, 273.59, 1.73]
9:[265.96 107.13   3.37] (10); Answer: [255.34, 66.63, 1.21]
10:[ 20.77 244.49   4.81] (11); Answer: [104.46, 203.54, 1.95]
11:[-57.4   -9.29   4.52] (13); Answer: [-120.39, 92.32, 1.72]
12:[ 20.77 244.49   4.81] (11); Answer: [114.52, 198.68, 1.33]
13:[-268.62 -100.39    4.84] (6); Answer: [-315.97, -70.5, 1.36]
14:[-273.67  -21.14    4.98] (7); Answer: [-257.8, 45.43, 1.32]
15:[  9.76 -28.57   2.24] (15); Answer: [-18.52, -30.78, 1

132:[-268.62 -100.39    4.84] (6); Answer: [-316.63, -68.15, 1.8]
133:[252.85 260.85   3.29] (6); Answer: [195.48, 321.3, 1.7]
134:[ 743.4  -125.45    0.88] (16); Answer: [-136.49, 9.58, 1.02]
135:[-268.62 -100.39    4.84] (4); Answer: [-314.56, -223.65, 1.83]
136:[-317.88  114.67    5.07] (7); Answer: [-241.97, 83.72, 1.99]
137:[  -3.92 -273.36    5.47] (6); Answer: [11.43, -276.89, 1.7]
138:[-317.88  114.67    5.07] (6); Answer: [-304.84, 120.8, 1.07]
139:[-194.46 -119.81    5.41] (9); Answer: [-226.19, -120.04, 1.69]
140:[  -3.92 -273.36    5.47] (4); Answer: [95.2, -309.8, 1.15]
141:[-268.62 -100.39    4.84] (6); Answer: [-267.07, -150.95, 1.78]
142:[-194.46 -119.81    5.41] (9); Answer: [-208.68, -205.34, 1.63]
143:Not enough valid TOA data! Available: 2
144:[-268.62 -100.39    4.84] (6); Answer: [-310.0, -212.64, 1.13]
145:[ 257.91 -212.53    2.66] (9); Answer: [177.87, -193.39, 1.67]
146:Not enough valid TOA data! Available: 3
147:[252.85 260.85   3.29] (6); Answer: [247.24, 263

392:[  -3.92 -273.36    5.47] (4); Answer: [95.5, -316.09, 1.76]
393:[-57.4   -9.29   4.52] (15); Answer: [-43.43, -18.57, 1.98]
394:[-222.06  318.47    2.12] (6); Answer: [-140.84, 208.03, 1.25]
395:[ -84.5  -184.89    4.58] (8); Answer: [-132.52, -260.87, 1.73]
396:[ -84.5  -184.89    4.58] (12); Answer: [-62.32, -191.01, 1.01]
397:[ 279.66 -128.13    3.98] (10); Answer: [224.83, -80.44, 1.99]
398:[  10.02 -119.33    5.39] (15); Answer: [1.39, -89.41, 1.45]
399:Not enough valid TOA data! Available: 2
400:Not enough valid TOA data! Available: 2
401:[252.85 260.85   3.29] (8); Answer: [156.89, 282.21, 1.41]
402:Not enough valid TOA data! Available: 2
403:[ 42.85 311.47   4.22] (7); Answer: [110.3, 303.83, 1.1]
404:[-268.62 -100.39    4.84] (6); Answer: [-307.45, -92.21, 1.68]
405:[-454.77  -24.58    6.76] (11); Answer: [-199.0, -108.19, 1.78]
406:[-156.24  -21.44    4.94] (16); Answer: [-64.92, -48.93, 1.91]
407:[-57.4   -9.29   4.52] (14); Answer: [-69.76, 70.9, 1.84]
408:[-317.88  11

652:[-1016.05    67.65     8.21] (15); Answer: [-84.87, -96.68, 1.41]
653:[252.85 260.85   3.29] (4); Answer: [274.9, 297.75, 1.01]
654:[-178.93  -51.22    2.26] (12); Answer: [-172.62, -77.6, 1.36]
655:[ 279.66 -128.13    3.98] (11); Answer: [182.87, -132.22, 1.58]
656:[-222.06  318.47    2.12] (5); Answer: [-170.47, 278.1, 1.06]
657:[ 279.66 -128.13    3.98] (6); Answer: [299.44, -111.75, 1.12]
658:[ -5.79 153.43   4.2 ] (9); Answer: [-80.48, 165.65, 1.23]
659:[ -84.5  -184.89    4.58] (9); Answer: [-101.23, -238.28, 1.84]
660:[ -5.79 153.43   4.2 ] (13); Answer: [-78.22, 92.41, 1.53]
661:[  -3.92 -273.36    5.47] (4); Answer: [104.8, -303.3, 1.54]
662:[-317.88  114.67    5.07] (5); Answer: [-235.01, 135.79, 1.61]
663:[ 257.91 -212.53    2.66] (7); Answer: [211.22, -191.13, 1.2]
664:[-222.06  318.47    2.12] (5); Answer: [-182.31, 280.26, 1.27]
665:[ -5.79 153.43   4.2 ] (13); Answer: [-41.48, 114.34, 2.0]
666:[166.94  -6.84   3.4 ] (16); Answer: [158.51, -2.88, 1.14]
667:[ 279.66 -1

907:[-273.67  -21.14    4.98] (10); Answer: [-239.71, -24.34, 1.01]
908:[275.77 421.68   6.56] (5); Answer: [297.94, 226.62, 1.49]
909:[166.94  -6.84   3.4 ] (14); Answer: [164.66, 9.3, 1.97]
910:[-194.46 -119.81    5.41] (9); Answer: [-207.77, -199.49, 1.85]
911:[ -5.79 153.43   4.2 ] (10); Answer: [-113.31, 166.48, 1.51]
912:Not enough valid TOA data! Available: 3
913:[166.94  -6.84   3.4 ] (14); Answer: [166.65, 39.97, 1.95]
914:Not enough valid TOA data! Available: 2
915:[ 36.29 -85.79   3.19] (16); Answer: [104.55, -87.35, 1.04]
916:Not enough valid TOA data! Available: 2
917:[-273.67  -21.14    4.98] (6); Answer: [-300.24, -35.02, 1.69]
918:[-57.4   -9.29   4.52] (16); Answer: [-106.73, -22.85, 1.05]
919:[-268.62 -100.39    4.84] (6); Answer: [-278.61, -127.85, 1.56]
920:[ -32.55 -110.14    2.77] (12); Answer: [-31.1, -151.22, 1.28]
921:[265.96 107.13   3.37] (8); Answer: [273.89, 159.68, 1.27]
922:Not enough valid TOA data! Available: 2
923:[ 20.77 244.49   4.81] (8); Answer: [6

1041:[ -5.79 153.43   4.2 ] (8); Answer: [-60.68, 184.19, 1.94]
1042:[  10.02 -119.33    5.39] (12); Answer: [63.34, -192.67, 1.87]
1043:[-268.62 -100.39    4.84] (4); Answer: [-310.07, -228.48, 1.09]
1044:[ 42.85 311.47   4.22] (8); Answer: [85.87, 277.04, 1.67]
1045:[-268.62 -100.39    4.84] (6); Answer: [-321.7, -77.06, 1.93]
1046:[ -5.79 153.43   4.2 ] (8); Answer: [-32.76, 132.09, 1.73]
1047:Not enough valid TOA data! Available: 3
1048:Not enough valid TOA data! Available: 3
1049:[ -5.79 153.43   4.2 ] (8); Answer: [-141.31, 189.71, 1.87]
1050:[ 36.29 -85.79   3.19] (14); Answer: [80.92, -73.85, 1.73]
1051:[ 42.85 311.47   4.22] (8); Answer: [91.6, 258.63, 1.77]
1052:[-57.4   -9.29   4.52] (17); Answer: [-114.43, 23.95, 1.19]
1053:[-268.62 -100.39    4.84] (6); Answer: [-260.17, -150.9, 1.96]
1054:[ 87.23 -13.2    3.86] (15); Answer: [113.73, 31.08, 1.33]
1055:[  -3.92 -273.36    5.47] (9); Answer: [23.59, -218.64, 1.83]
1056:Not enough valid TOA data! Available: 2
1057:[-194.46 -